In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from scipy import ndimage, misc
import cv2
from skimage.morphology import skeletonize
from fil_finder import FilFinder2D
import astropy.units as u
import gc
import sklearn.metrics
gc.collect()


In [ ]:
img =cv2.imread('BMS-tile DAPI annotated eroded.tif') #cv2.imread("BMS-tile DAPI annotated.tif")
img_gene1=cv2.imread('BMS-tile_gene1.tif')
img_gene2=cv2.imread('BMS-tile_gene2.tif')

In [ ]:
def get_gene_dipole(hey,contour,epsilon,img_gene1,img_gene2): #function that separates organoids into sections and returns gene dipole
    jj=0
    
    #n_segments=20 #number of segments
    #epsilon=np.shape(hey)[0]//n_segments
    
    epsilon=epsilon
    n_segments=np.shape(hey)[0]//epsilon
    initial_pos=epsilon
    gene1=np.empty([1,0])
    gene2=np.empty([1,0])
    position=np.empty([1,0])
    while (initial_pos+epsilon)<(np.shape(hey)[0]): #iterate through every point on skeleton to get normal lines
        
        x_vec=hey[:,0]
        y_vec=hey[:,1]
        fit=np.polyfit(x=x_vec[initial_pos-epsilon:initial_pos+epsilon], y=y_vec[initial_pos-epsilon:initial_pos+epsilon], deg=1)
        m_normal=-1/fit[0]
        b_normal=y_vec[initial_pos]-(m_normal*x_vec[initial_pos])
        
        if abs(m_normal)>1000:
            y_list=np.linspace(y_vec[initial_pos]-100,y_vec[initial_pos]+100,100000)
            x_list=np.repeat(x_vec[initial_pos], np.shape(y_list)[0])
            y_list = [int(x) for x in y_list]
            x_list=[int(x) for x in x_list]
        else:
            x_list=np.linspace(x_vec[initial_pos]-100,x_vec[initial_pos]+100,100000)
            y_list=m_normal*x_list+b_normal
            y_list = [int(x) for x in y_list]
            x_list=[int(x) for x in x_list]
        coord_list=list(zip(y_list, x_list))
        arr2_2=np.array(coord_list)
        arr1=np.squeeze(contour)
        for hh in range(np.shape(arr2_2)[0]-1):
            if arr2_2[hh+1,0]==arr2_2[hh,0]+1 and arr2_2[hh+1,1]==arr2_2[hh,1]+1:
                temp=np.append(arr2_2[0:hh+1,:],[arr2_2[hh,:]+[1,0]],axis=0)
                arr2_2=np.append(temp,arr2_2[hh+1:],axis=0)
            if arr2_2[hh+1,0]==arr2_2[hh,0]-1 and arr2_2[hh+1,1]==arr2_2[hh,1]+1:
                temp=np.append(arr2_2[0:hh+1,:],[arr2_2[hh,:]-[1,0]],axis=0)
                arr2_2=np.append(temp,arr2_2[hh+1:],axis=0)
        
        intersection=np.array([x for x in set(tuple(x) for x in arr2_2) & set(tuple(x) for x in arr1)])
        if np.shape(intersection)[0]<2:
            arr1=np.concatenate((arr1,arr1+1),axis=0)
            intersection=np.array([x for x in set(tuple(x) for x in arr2_2) & set(tuple(x) for x in arr1)])
        if np.shape(intersection)[0]>2: #fix this make more general
            dist_1=sklearn.metrics.pairwise_distances(intersection)
            np.fill_diagonal(dist_1, 100)
            dist_1=np.tril(dist_1, k=0)
            dist_1[dist_1==0]=100
            a=np.where(dist_1<5)
            mid_point=np.delete(intersection, a[0][:],0)
            intersection=mid_point
            if np.shape(mid_point)[0]>2:
                repe=np.repeat([[y_vec[initial_pos],x_vec[initial_pos]]],np.shape(mid_point)[0], axis=0)
                dist=np.sqrt((mid_point[:,0]-repe[:,0])**2+(mid_point[:,1]-repe[:,1])**2)
                mins=np.partition(dist,2)[0:2]
                min_id1=np.where(dist==mins[0])
                min_id2=np.where(dist==mins[1])
                intersection=mid_point[[min_id1[0][0],min_id2[0][0]],:]
        
        id1=np.where((arr2_2[:,1]==intersection[0,1]) & (arr2_2[:,0]==intersection[0,0]))
        id2=np.where((arr2_2[:,1]==intersection[1,1]) & (arr2_2[:,0]==intersection[1,0]))
        id_min=np.min([id1[0][0],id2[0][0]])
        id_max=np.max([id1[0][0],id2[0][0]])
        side1=coord_list[id_min:id_max]
    
        edge1=np.where((arr1[:,1]==intersection[0,1]) & (arr1[:,0]==intersection[0,0]))
        edge2=np.where((arr1[:,1]==intersection[1,1]) & (arr1[:,0]==intersection[1,0]))
        #edge_min_1=np.min([edge1,edge2])
        #edge_max_2=np.max([edge1,edge2])
    
        
        
        
        initial_pos=initial_pos+epsilon
        
        
        x_vec=hey[:,0]
        y_vec=hey[:,1]
        fit=np.polyfit(x=x_vec[initial_pos-epsilon:initial_pos+epsilon], y=y_vec[initial_pos-epsilon:initial_pos+epsilon], deg=1)
        m_normal=-1/fit[0]
        b_normal=y_vec[initial_pos]-(m_normal*x_vec[initial_pos])
        
        if abs(m_normal)>1000:
            y_list=np.linspace(y_vec[initial_pos]-100,y_vec[initial_pos]+100,100000)
            x_list=np.repeat(x_vec[initial_pos], np.shape(y_list)[0])
            y_list = [int(x) for x in y_list]
            x_list=[int(x) for x in x_list]
        else:
            x_list=np.linspace(x_vec[initial_pos]-100,x_vec[initial_pos]+100,100000)
            y_list=m_normal*x_list+b_normal
            y_list = [int(x) for x in y_list]
            x_list=[int(x) for x in x_list]
        coord_list=list(zip(y_list, x_list))
        arr2=np.array(coord_list)
        arr1=np.squeeze(contour)
        
        for hh in range(np.shape(arr2)[0]-1):
            if arr2[hh+1,0]==arr2[hh,0]+1 and arr2[hh+1,1]==arr2[hh,1]+1:
                temp=np.append(arr2[0:hh+1,:],[arr2[hh,:]+[1,0]],axis=0)
                arr2=np.append(temp,arr2[hh+1:],axis=0)
            if arr2[hh+1,0]==arr2[hh,0]-1 and arr2[hh+1,1]==arr2[hh,1]+1:
                temp=np.append(arr2[0:hh+1,:],[arr2[hh,:]+[1,0]],axis=0)
                arr2=np.append(temp,arr2[hh+1:],axis=0)
                
                
       
        intersection=np.array([x for x in set(tuple(x) for x in arr2) & set(tuple(x) for x in arr1)])
        if np.shape(intersection)[0]<2:
            arr1=np.concatenate((arr1,arr1+1),axis=0)
            intersection=np.array([x for x in set(tuple(x) for x in arr2) & set(tuple(x) for x in arr1)])
        if np.shape(intersection)[0]>2: #fix this make more general
            dist_1=sklearn.metrics.pairwise_distances(intersection)
            np.fill_diagonal(dist_1, 100)
            dist_1=np.tril(dist_1, k=0)
            dist_1[dist_1==0]=100
            a=np.where(dist_1<5)
            mid_point=np.delete(intersection, a[0][:],0)
            intersection=mid_point
            if np.shape(mid_point)[0]>2:
                repe=np.repeat([[y_vec[initial_pos],x_vec[initial_pos]]],np.shape(mid_point)[0], axis=0)
                dist=np.sqrt((mid_point[:,0]-repe[:,0])**2+(mid_point[:,1]-repe[:,1])**2)
                mins=np.partition(dist,2)[0:2]
                min_id1=np.where(dist==mins[0])
                min_id2=np.where(dist==mins[1])
                intersection=mid_point[[min_id1[0][0],min_id2[0][0]],:]     
        
        id1=np.where((arr2[:,1]==intersection[0,1]) & (arr2[:,0]==intersection[0,0]))
        id2=np.where((arr2[:,1]==intersection[1,1]) & (arr2[:,0]==intersection[1,0]))
        id_min=np.min([id1[0][0],id2[0][0]])
        id_max=np.max([id1[0][0],id2[0][0]])
        side2=coord_list[id_min:id_max]
        

        edge3=np.where((arr1[:,1]==intersection[0,1]) & (arr1[:,0]==intersection[0,0]))
        edge4=np.where((arr1[:,1]==intersection[1,1]) & (arr1[:,0]==intersection[1,0]))
        #edge_min_3=np.min([edge3,edge4])
        #edge_max_4=np.max([edge3,edge4])
        edges=np.sort([edge1[0][0],edge2[0][0],edge3[0][0],edge4[0][0]])
        if edges[0]==edges[1]:
            edges[1]=edges[1]+1
        if edges[2]==edges[3]:
            edges[3]=edges[3]+1
        line1=arr1[edges[0]:edges[1]]
        line2=arr1[edges[2]:edges[3]]
        if np.sum(line1[0,:]-line1[-1,:]) <0:
            line1=np.flip(line1,0)
        if np.sum(line2[0,:]-line2[-1,:]) >0:
            line2=np.flip(line2,0)
        side2=np.flip(side2,0)
        region_cont=np.concatenate((line1,side1,line2,side2))
        side_cross=np.array([x for x in set(tuple(x) for x in side1) & set(tuple(x) for x in side2)])
        if np.shape(side_cross)[0]>0:
            line1=np.flip(line1,0)
            line2=np.flip(line2,0)
        im_zer = np.zeros( (img.shape[0],img.shape[1]) ) # create a single channel 200x200 pixel black image 
        filled_array = cv2.fillPoly(im_zer, pts =[region_cont], color=(255))

        region = np.argwhere(filled_array == 255)
        segment=np.squeeze(img_gene1[:,:,0])*filled_array
        segment[segment == 0] = np.nan
        gene1=np.append(gene1, np.nanmean(segment)/255)
        segment=np.squeeze(img_gene2[:,:,0])*filled_array
        segment[segment == 0] = np.nan
        gene2=np.append(gene2, np.nanmean(segment)/255)
        position=np.append(position, initial_pos-epsilon/2)
        initial_pos=initial_pos
         
    if np.shape(gene1)[0]==1:
        area_red=gene1[0]
        area_blue=gene2[0]
    else:
        area_red = np.trapz(gene1/np.max(gene1), dx=position[1]-position[0])
        area_blue = np.trapz(gene2/np.max(gene1), dx=position[1]-position[0])
        
    dipole=area_red/(area_red+area_blue)
    if dipole.size == 0:
        dipole=0
    return dipole

In [ ]:
# Grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(img, 30, 200)

contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
start = timeit.timeit()
dipole_list=np.empty([1,0])
true_org=np.empty([1,0])

for ii in range(np.shape(contours)[0]):
    
    M = cv2.moments(contours[ii])
    if M["m00"] == 0:
        true_org=np.append(true_org,0)
    else:
        true_org=np.append(true_org,1)


for ii in range(np.shape(contours)[0]):

    if true_org[ii]==1:
        im_zer = np.zeros( (img.shape[0],img.shape[1]) ) # create a single channel 200x200 pixel black image 
        filled_array = cv2.fillPoly(im_zer, pts =[contours[ii]], color=(255))

        obj = np.argwhere(filled_array == 255)

        

        bin_arr = np.where(filled_array > 0, 1, 0)
    
        image = bin_arr

    # perform skeletonization
    
        skeleton1 = skeletonize(image)


        binSkel = skeleton1*1
        skelObj = np.argwhere(binSkel == 1)

        fil = FilFinder2D(binSkel, distance=250 * u.pc, mask=skeleton1)
        fil.preprocess_image(flatten_percent=85)
        fil.create_mask(border_masking=True, verbose=False,
        use_existing_mask=True)
        fil.medskel(verbose=False)
        fil.analyze_skeletons(branch_thresh=40* u.pix, skel_thresh=10 * u.pix, prune_criteria='length')
        skeleton_length=fil.lengths()
        plt.plot(obj[:,0],obj[:,1], 'bo')
        plt.plot(skelObj[:,0],skelObj[:,1], 'ro')


        x,y,w,h = cv2.boundingRect(contours[ii])
        aspect_ratio = float(w)/h
        M = cv2.moments(contours[ii])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centroid=np.array([cY,cX])
        hey_1 = np.argwhere(fil.skeleton_longpath==1)
        if hey_1.size==0:
            hey_1=skelObj
        im_zer = np.zeros( (img.shape[0],img.shape[1]) )
        skele_im=im_zer
        skele_im[hey_1[:,0],hey_1[:,1]]=255
        n=0
        ma=np.array([1,1,1])
        ok2=0
        while ok2==0:
            
            in_x=hey_1[n,0]
            in_y=hey_1[n,1]
            ma=skele_im[in_x-1:in_x+2,in_y-1:in_y+2]
            n=n+1
            if np.sum(ma)<255*3:
                ok2=1
        skeleton=np.array([[0,1],[in_x, in_y]])
        ok=0
        x_prev=1
        y_prev=1
        neighbors=skele_im[in_x-1:in_x+2,in_y-1:in_y+2]
        center_y=in_y
        center_x=in_x
        n=0
        while ok==0:
        
        
            neighbors[x_prev,y_prev]=0
            neighbors[1,1]=0
            res=np.where(neighbors==255)
            if res[0].size==0 :
         
                break
        
            y_next=res[1]
            x_next=res[0]
            skeleton=np.append(skeleton, [[x_next[0]-1+center_x, y_next[0]-1+center_y]], axis=0)
            y_prev=2-y_next[0]
            x_prev=2-x_next[0]
        
            center_y=center_y+y_next[0]-1
            center_x=center_x+x_next[0]-1
        
            neighbors=skele_im[-1+center_x:2+center_x,-1+center_y:2+center_y]
        

        skeleton=np.delete(skeleton, 0,0)
        hey=skeleton
        epsilon=20
    
        dipole=get_gene_dipole(hey,contours[ii],epsilon,img_gene1,img_gene2)
        dipole_list=np.append(dipole_list,dipole)
        print('dipole gene=',dipole)
        print('elongation=',skeleton_length)
        print('aspect ratio=',aspect_ratio)
        print('centroid=',centroid)
end = timeit.timeit()
print(end - start)